In [1]:
import bokeh
import matplotlib as plt
import numpy as np
%matplotlib inline
import pandas as pd

from bokeh.models.widgets import Button, Paragraph, RadioButtonGroup, Select, Slider
from bokeh.models import Select
from bokeh.io import show, output_notebook, output_file
from bokeh.plotting import curdoc
output_notebook()

Loading BokehJS ...

In [19]:
import json
import pps_regressor

json_out = 'selection.json'
def update_data(event):
    bldg_data = {
        'Neighborhood': str(nbhd.value_input),
        'Building-Type-Custom': density.active,
        'Fixtures': fixtures.value,
        'Year Built': int(yearbuilt.value_input),
        'Building-Grade': segment.active,
        'Square Footage': squarefootage.value,
        'Renovation': reno_new.active,
        'Quarter': quarter.active,
        'Year': 2019}
    with open(json_out, 'w') as out:
        json.dump(bldg_data, out)
    price_pred = pps_regressor.prediction(json_out)
    floatpred = float(price_pred)
    with open('prediction.json', 'w') as file:
        json.dump(floatpred, file)
    p = Paragraph(text="{}".format(price_pred * bldg_data['Square Footage']))
    show(p)

In [22]:
from bokeh.layouts import column, widgetbox
from bokeh.models.widgets import Dropdown, Slider, TextInput, CheckboxButtonGroup, Select, RadioButtonGroup, RadioGroup

# Create some widgets
nbhd = TextInput(placeholder="4-digit #", title="Neighborhood Code:")
densities = {"Single-Family":"SINGLE FAM", "Plex": "PLEX", "Mid-Rise": "CONDO", "High-Rise": "HRISE CONDO"}
density = RadioButtonGroup(labels=['Single-Family', 'Plex', 'Mid-Rise', 'High-Rise'])
reno_new = RadioButtonGroup(labels=["Renovation", "New Build"])
#If renovation, select year built of the property:
yearbuilt = TextInput(placeholder="4-digit #", title="Year of the original structure (if renovation):")
squarefootage = Slider(start=500, end=3500, value=500, step=10, title="Square Footage")
segments = {"Affordable": "C", "Mid-Range":"B", "High-End":"A", "Ultra-Lux":"X"}
segment = RadioButtonGroup(
        labels=["Affordable", "Mid-Range", "High-End", "Luxury"])
quarters = [("One", 1), ("Two", 2), ("Three", 3), ("Four", 4)]
quarter = RadioGroup(labels = ['First', 'Second', 'Third', 'Fourth'])
fixtures = Slider(start=3, end=20, value=3, step=1, title="Water Fixtures")
calculate = Button(label="Get Price", button_type="success")
calculate.on_click(update_data)
box = widgetbox([density, nbhd, reno_new, yearbuilt, squarefootage, segment, fixtures, quarter, calculate])

In [23]:
def show_box(doc):
    doc.add_root(box)
show(show_box)

ERROR:bokeh.server.protocol_handler:error handling message Message 'EVENT' (revision 1) content: '{"event_name":"button_click","event_values":{"model_id":"1106"}}': NameError("name 'price_pred' is not defined")


0    83247.422464
Name: Square Footage, dtype: float64


ERROR:bokeh.server.protocol_handler:error handling message Message 'EVENT' (revision 1) content: '{"event_name":"button_click","event_values":{"model_id":"1106"}}': NameError("name 'price_pred' is not defined")


0    83247.422464
Name: Square Footage, dtype: float64


In [25]:
import numpy as np
import pandas as pd 
import lightgbm as lgb
import json

DENSITIES = {0:"SINGLE FAM", 1: "PLEX", 2: "CONDO", 3: "HRISE CONDO"}
SEGMENTS = {0: 'C', 1: 'B', 2: 'A', 3:'X'}
model = lgb.Booster(model_file='pps-model.txt')
with open('selection.json') as testfile:
    bokeh_dict = json.load(testfile)
lst = [dict(bokeh_dict)]
tester = pd.DataFrame(lst)
#Converting json dataset from bokeh to dataframe
tester['Log-Built'] = np.log(tester['Year Built'])
tester['Log-Fixtures'] = np.log(int(tester['Fixtures']))
tester['Log-SqFt'] = np.log(int(tester['Square Footage']))
tester['Neighborhood'] = tester['Neighborhood'].astype(float)
with open('meanpps.json', 'r') as nbhdpps:
    nbhd_dict = json.load(nbhdpps)
meanpps = pd.DataFrame(nbhd_dict, index=None)
meanpps.index=meanpps.index.astype(float)
nbhds = list(meanpps.index)
tester=pd.merge(how = 'left', left = tester, right = meanpps,left_on="Neighborhood", right_index=True)
tester['Log-NbhdPPS'] = np.log(tester['Neighborhood'])
tester['Neighborhood'] = str(tester['Neighborhood'])
# tester['Quarter'] = tester['Quarter'].map(lambda x: QUARTERS[x])
tester['Building-Grade'] = tester['Building-Grade'].map(lambda x: SEGMENTS[x])
tester['Building-Type-Custom'] = tester['Building-Type-Custom'].map(lambda x: DENSITIES[x])
X = pd.get_dummies(tester[['Year', 'Log-Built', 'Neighborhood', 'Log-Fixtures', 
                           'Building-Type-Custom', 'Quarter', 'Building-Grade', 
                           'Log-SqFt', 'Log-NbhdPPS']])
price_pred = model.predict(X)
# print(price_pred * tester['Square Footage'])

In [26]:
from bokeh.models.glyphs import Text
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid
from bokeh.models.widgets import Paragraph
bldg_data = {
        'Neighborhood': str(nbhd.value_input),
        'Building-Type-Custom': density.active,
        'Fixtures': fixtures.value,
        'Year Built': int(yearbuilt.value_input),
        'Building-Grade': segment.active,
        'Square Footage': squarefootage.value,
        'Renovation': reno_new.active,
        'Quarter': quarter.active,
        'Year': 2019}
p = Paragraph(text="{}".format(price_pred * bldg_data['Square Footage']))
show(p)


In [47]:

with open('prediction.json', 'r') as file:
    pred = json.load(file)
p = Paragraph(text="{}".format(price_pred * bldg_data['Square Footage']))

show(p)

In [45]:
floatpred = float(price_pred)
with open('prediction.json', 'w') as file:
    json.dump(floatpred, file)